In [ ]:
import torch
import torch.nn as nn
import torch.nn.parallel
from __future__ import print_function
import argparse
import random
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.optim as optim
import torch.utils.data
import torchvision.datasets as dset
import torchvision.transforms as transforms
import torchvision.utils as vutils
from torch.autograd import Variable
import os

batchsize=64

class DCGAN_D(nn.Module):
    def __init__(self, isize=32, nz=100, nc=3, ndf=64, ngpu=0, n_extra_layers=0):
        super(DCGAN_D, self).__init__()
        self.ngpu = ngpu
        assert isize % 16 == 0, "isize has to be a multiple of 16"

        main = nn.Sequential()
        # input is nc x isize x isize
        main.add_module('initial.conv.{0}-{1}'.format(nc, ndf),
                        nn.Conv2d(nc, ndf, 4, 2, 1, bias=False))
        main.add_module('initial.relu.{0}'.format(ndf),
                        nn.LeakyReLU(0.2, inplace=True))
        csize, cndf = isize / 2, ndf

        # Extra layers
        for t in range(n_extra_layers):
            main.add_module('extra-layers-{0}.{1}.conv'.format(t, cndf),
                            nn.Conv2d(cndf, cndf, 3, 1, 1, bias=False))
            main.add_module('extra-layers-{0}.{1}.batchnorm'.format(t, cndf),
                            nn.BatchNorm2d(cndf))
            main.add_module('extra-layers-{0}.{1}.relu'.format(t, cndf),
                            nn.LeakyReLU(0.2, inplace=True))

        while csize > 4:
            in_feat = cndf
            out_feat = cndf * 2
            main.add_module('pyramid.{0}-{1}.conv'.format(in_feat, out_feat),
                            nn.Conv2d(in_feat, out_feat, 4, 2, 1, bias=False))
            main.add_module('pyramid.{0}.batchnorm'.format(out_feat),
                            nn.BatchNorm2d(out_feat))
            main.add_module('pyramid.{0}.relu'.format(out_feat),
                            nn.LeakyReLU(0.2, inplace=True))
            cndf = cndf * 2
            csize = csize / 2

        # state size. K x 4 x 4
        main.add_module('final.{0}-{1}.conv'.format(cndf, 1),
                        nn.Conv2d(cndf, 1, 4, 1, 0, bias=False))
        self.main = main


    def forward(self, input):
        if isinstance(input.data, torch.cuda.FloatTensor) and self.ngpu > 1:
            output = nn.parallel.data_parallel(self.main, input, range(self.ngpu))
        else: 
            output = self.main(input)
            
        output = output.mean(0)
        return output.view(1)

class DCGAN_G(nn.Module):
    def __init__(self, isize=32, nz=100, nc=3, ngf=64, ngpu=0, n_extra_layers=0):
        super(DCGAN_G, self).__init__()
        self.ngpu = ngpu
        assert isize % 16 == 0, "isize has to be a multiple of 16"

        cngf, tisize = ngf//2, 4
        while tisize != isize:
            cngf = cngf * 2
            tisize = tisize * 2

        main = nn.Sequential()
        # input is Z, going into a convolution
        main.add_module('initial.{0}-{1}.convt'.format(nz, cngf),
                        nn.ConvTranspose2d(nz, cngf, 4, 1, 0, bias=False))
        main.add_module('initial.{0}.batchnorm'.format(cngf),
                        nn.BatchNorm2d(cngf))
        main.add_module('initial.{0}.relu'.format(cngf),
                        nn.ReLU(True))

        csize, cndf = 4, cngf
        while csize < isize//2:
            main.add_module('pyramid.{0}-{1}.convt'.format(cngf, cngf//2),
                            nn.ConvTranspose2d(cngf, cngf//2, 4, 2, 1, bias=False))
            main.add_module('pyramid.{0}.batchnorm'.format(cngf//2),
                            nn.BatchNorm2d(cngf//2))
            main.add_module('pyramid.{0}.relu'.format(cngf//2),
                            nn.ReLU(True))
            cngf = cngf // 2
            csize = csize * 2

        # Extra layers
        for t in range(n_extra_layers):
            main.add_module('extra-layers-{0}.{1}.conv'.format(t, cngf),
                            nn.Conv2d(cngf, cngf, 3, 1, 1, bias=False))
            main.add_module('extra-layers-{0}.{1}.batchnorm'.format(t, cngf),
                            nn.BatchNorm2d(cngf))
            main.add_module('extra-layers-{0}.{1}.relu'.format(t, cngf),
                            nn.ReLU(True))

        main.add_module('final.{0}-{1}.convt'.format(cngf, nc),
                        nn.ConvTranspose2d(cngf, nc, 4, 2, 1, bias=False))
        main.add_module('final.{0}.tanh'.format(nc),
                        nn.Tanh())
        self.main = main

    def forward(self, input):
        if isinstance(input.data, torch.cuda.FloatTensor) and self.ngpu > 1:
            output = nn.parallel.data_parallel(self.main, input, range(self.ngpu))
        else: 
            output = self.main(input)
        return output 
###############################################################################

In [ ]:
import cv2
import numpy as np
batchSize = 64

def bonemarrow_cell():
    X = np.load("/home/msragpu/cellwork/data/data.npy")
    img = X
    X = np.asarray([cv2.resize(x, (32,32)) for x in X])
    X = np.asarray([x[:,:,::-1].transpose((2,0,1)) for x in X])
    X = X.astype(np.float32)/(255.0/2) - 1.0
    return X

X_train = bonemarrow_cell()
X_label = torch.LongTensor(np.zeros((X_train.shape[0]),dtype=int))
X_train = torch.FloatTensor(X_train)
train = torch.utils.data.TensorDataset(X_train,X_label)
dataloader = torch.utils.data.DataLoader(train, shuffle=True, batch_size=batchsize)

dataiter = iter(dataloader)
ngpu = 0
nz = 100
ngf = 64
ndf = 64
nc = 3
n_extra_layers = 0

# custom weights initialization called on netG and netD
def weights_init(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        m.weight.data.normal_(0.0, 0.02)
    elif classname.find('BatchNorm') != -1:
        m.weight.data.normal_(1.0, 0.02)
        m.bias.data.fill_(0)

netG = DCGAN_G()
netG.apply(weights_init)
print(netG)

netD = DCGAN_D()
netD.apply(weights_init)
print(netD)

input = torch.FloatTensor(batchSize, 3, 32, 32)
noise = torch.FloatTensor(batchSize, nz, 1, 1)
fixed_noise = torch.FloatTensor(batchSize, nz, 1, 1).normal_(0, 1)
one = torch.FloatTensor([1])
mone = one * -1

netD.cuda()
netG.cuda()
input = input.cuda()
one, mone = one.cuda(), mone.cuda()
noise, fixed_noise = noise.cuda(), fixed_noise.cuda()

optimizerD = optim.RMSprop(netD.parameters(), lr = 0.00005)
optimizerG = optim.RMSprop(netG.parameters(), lr = 0.00005)

gen_iterations = 0
for epoch in range(10000):
    data_iter = iter(dataloader)
    i = 0
    while i < len(dataloader):
        ############################
        # (1) Update D network
        ###########################
        for p in netD.parameters(): # reset requires_grad
            p.requires_grad = True # they are set to False below in netG update

        # train the discriminator Diters times
        if gen_iterations < 25 or gen_iterations % 500 == 0:
            Diters = 100
        else:
            Diters = 5
        j = 0
        while j < Diters and i < len(dataloader):
            j += 1

            # clamp parameters to a cube
            for p in netD.parameters():
                p.data.clamp_(-0.01, 0.01)

            data = data_iter.next()
            i += 1

            # train with real
            real_cpu, _ = data
            netD.zero_grad()
            batch_size = real_cpu.size(0)

            real_cpu = real_cpu.cuda()
            input.resize_as_(real_cpu).copy_(real_cpu)
            inputv = Variable(input)

            errD_real = netD(inputv)
            errD_real.backward(one)

            # train with fake
            noise.resize_(64, 100, 1, 1).normal_(0, 1)
            noisev = Variable(noise, volatile = True) # totally freeze netG
            fake = Variable(netG(noisev).data)
            inputv = fake
            errD_fake = netD(inputv)
            errD_fake.backward(mone)
            errD = errD_real - errD_fake
            optimizerD.step()

        ############################
        # (2) Update G network
        ###########################
        for p in netD.parameters():
            p.requires_grad = False # to avoid computation
        netG.zero_grad()
        # in case our last batch was the tail batch of the dataloader,
        # make sure we feed a full batch of noise
        noise.resize_(64, 100, 1, 1).normal_(0, 1)
        noisev = Variable(noise)
        fake = netG(noisev)
        errG = netD(fake)
        errG.backward(one)
        optimizerG.step()
        gen_iterations += 1


        if gen_iterations % 20 == 0:
            
            print('[%d/%d][%d/%d][%d] Loss_D: %f Loss_G: %f Loss_D_real: %f Loss_D_fake %f'
            % (epoch, 10000, i, len(dataloader), gen_iterations,
            errD.data[0], errG.data[0], errD_real.data[0], errD_fake.data[0]))
            real_cpu = real_cpu.mul(0.5).add(0.5)
            vutils.save_image(real_cpu, 'real_samples.png')
            fake = netG(Variable(fixed_noise, volatile=True))
            fake.data = fake.data.mul(0.5).add(0.5)
            vutils.save_image(fake.data, 'fake_samples_1.png')

DCGAN_G (
  (main): Sequential (
    (initial.100-256.convt): ConvTranspose2d(100, 256, kernel_size=(4, 4), stride=(1, 1), bias=False)
    (initial.256.batchnorm): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True)
    (initial.256.relu): ReLU (inplace)
    (pyramid.256-128.convt): ConvTranspose2d(256, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (pyramid.128.batchnorm): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True)
    (pyramid.128.relu): ReLU (inplace)
    (pyramid.128-64.convt): ConvTranspose2d(128, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (pyramid.64.batchnorm): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True)
    (pyramid.64.relu): ReLU (inplace)
    (final.64-3.convt): ConvTranspose2d(64, 3, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (final.3.tanh): Tanh ()
  )
)
DCGAN_D (
  (main): Sequential (
    (initial.conv.3-64): Conv2d(3, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1)

[125/10000][35/61][1320] Loss_D: -0.118510 Loss_G: -0.012028 Loss_D_real: 0.024256 Loss_D_fake 0.142765
[127/10000][5/61][1340] Loss_D: -0.114992 Loss_G: 0.076524 Loss_D_real: -0.063968 Loss_D_fake 0.051024
[128/10000][40/61][1360] Loss_D: -0.122773 Loss_G: 0.108266 Loss_D_real: -0.078519 Loss_D_fake 0.044253
[130/10000][10/61][1380] Loss_D: -0.126638 Loss_G: 0.013695 Loss_D_real: -0.059307 Loss_D_fake 0.067331
[131/10000][45/61][1400] Loss_D: -0.113202 Loss_G: 0.084305 Loss_D_real: -0.045928 Loss_D_fake 0.067274
[133/10000][15/61][1420] Loss_D: -0.110331 Loss_G: 0.026579 Loss_D_real: -0.036456 Loss_D_fake 0.073875
[134/10000][50/61][1440] Loss_D: -0.117054 Loss_G: 0.105128 Loss_D_real: -0.073063 Loss_D_fake 0.043991
[136/10000][20/61][1460] Loss_D: -0.132143 Loss_G: 0.037141 Loss_D_real: -0.053850 Loss_D_fake 0.078293
[137/10000][55/61][1480] Loss_D: -0.116861 Loss_G: 0.030940 Loss_D_real: -0.031243 Loss_D_fake 0.085618
[139/10000][25/61][1500] Loss_D: -0.132910 Loss_G: 0.140849 Loss_

[248/10000][45/61][2900] Loss_D: -0.098372 Loss_G: 0.073065 Loss_D_real: -0.018290 Loss_D_fake 0.080082
[250/10000][15/61][2920] Loss_D: -0.112909 Loss_G: 0.052201 Loss_D_real: -0.025748 Loss_D_fake 0.087161
[251/10000][50/61][2940] Loss_D: -0.104802 Loss_G: 0.143292 Loss_D_real: -0.099527 Loss_D_fake 0.005275
[253/10000][20/61][2960] Loss_D: -0.086962 Loss_G: 0.027913 Loss_D_real: -0.000293 Loss_D_fake 0.086668
[254/10000][55/61][2980] Loss_D: -0.087279 Loss_G: 0.031786 Loss_D_real: -0.004654 Loss_D_fake 0.082625
[256/10000][25/61][3000] Loss_D: -0.110896 Loss_G: 0.076489 Loss_D_real: -0.042811 Loss_D_fake 0.068086
[258/10000][30/61][3020] Loss_D: -0.096443 Loss_G: -0.001696 Loss_D_real: -0.001581 Loss_D_fake 0.094862
[259/10000][61/61][3040] Loss_D: -0.040640 Loss_G: -0.076623 Loss_D_real: 0.072774 Loss_D_fake 0.113414
[261/10000][35/61][3060] Loss_D: -0.104652 Loss_G: 0.033564 Loss_D_real: -0.030131 Loss_D_fake 0.074522
[263/10000][5/61][3080] Loss_D: -0.068258 Loss_G: 0.018460 Loss

[371/10000][55/61][4480] Loss_D: -0.090223 Loss_G: 0.085669 Loss_D_real: -0.028831 Loss_D_fake 0.061392
[373/10000][25/61][4500] Loss_D: -0.094823 Loss_G: 0.106182 Loss_D_real: -0.070323 Loss_D_fake 0.024500
[375/10000][30/61][4520] Loss_D: -0.106764 Loss_G: 0.119487 Loss_D_real: -0.075698 Loss_D_fake 0.031067
[376/10000][61/61][4540] Loss_D: -0.039616 Loss_G: 0.001329 Loss_D_real: 0.037938 Loss_D_fake 0.077553
[378/10000][35/61][4560] Loss_D: -0.100652 Loss_G: 0.015681 Loss_D_real: 0.001046 Loss_D_fake 0.101699
[380/10000][5/61][4580] Loss_D: -0.068679 Loss_G: 0.091053 Loss_D_real: 0.014561 Loss_D_fake 0.083241
[381/10000][40/61][4600] Loss_D: -0.092518 Loss_G: -0.028156 Loss_D_real: -0.003693 Loss_D_fake 0.088826
[383/10000][10/61][4620] Loss_D: -0.093178 Loss_G: 0.082780 Loss_D_real: -0.069326 Loss_D_fake 0.023853
[384/10000][45/61][4640] Loss_D: -0.102293 Loss_G: 0.034158 Loss_D_real: -0.028415 Loss_D_fake 0.073878
[386/10000][15/61][4660] Loss_D: -0.103066 Loss_G: -0.006705 Loss_D